In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

import nltk
from nltk.corpus import wordnet as wn
import inflect

from keras.models import Sequential
from keras import layers
import numpy as np
from collections import Counter, defaultdict

from gensim.utils import tokenize
from itertools import groupby

from keras.models import Input, Model
from keras.layers import Dense, Dropout
from keras.layers import LSTM, RepeatVector
from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.


In [2]:
p = inflect.engine()

In [3]:
pairs = {}
for synset in wn.all_synsets('n'):
    word = synset.name().split('.', 1)[0]
    if not word in pairs:
        pairs[word] = p.plural(word)
len(pairs)

67176

In [4]:
with open('data/plurals.txt', 'w') as fout:
    for k in sorted(pairs):
        if '_' in k or '-' in k:
            continue
        if k.isdigit():
            continue
        fout.write('%s\t%s\n' % (k, pairs[k]))

In [5]:
p.plural('no')

'noes'

In [6]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [7]:
# Parameters for the model and dataset.
INVERT = True

In [8]:
questions = []
expected = []
seen = set()
#with open('data/en_de.txt') as fin:
with open('data/plurals.txt') as fin:
    for line in fin:
        en, de = line.strip().split('\t')
        questions.append(en)
        expected.append(de)

max_question_len = max(len(q) for q in questions)
max_expected_len = max(len(e) for e in expected)
questions = [' ' * (max_question_len - len(q)) + q for q in questions]
expected = [e + ' ' * (max_expected_len - len(e)) for e in expected]
if INVERT:
    questions = [q[::-1] for q in questions]

print('Total addition questions:', len(questions))

Total addition questions: 39929


In [9]:
chars = set(ch for k, v in zip(questions, expected) for ch in k + v)
ctable = CharacterTable(chars)
len(chars)

40

In [10]:
print('Vectorization...')
x = np.zeros((len(questions), max_question_len, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), max_expected_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, max_question_len)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, max_expected_len)
print('done')

Vectorization...
done


In [11]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Training Data:
(35937, 31, 40)
(35937, 32, 40)
Validation Data:
(3992, 31, 40)
(3992, 32, 40)


In [12]:
# The below is taken from: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(max_question_len, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.Dropout(DROP_OUT))
model.add(layers.RepeatVector(max_expected_len))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
#    model.add(layers.Dropout(DROP_OUT))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               86528     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 32, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 40)            5160      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 40)            0         
Total params: 223,272
Trainable params: 223,272
Non-trainable params: 0
_________________________________________________________________


In [17]:
BATCH_SIZE = 2048

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    print()
    print('-' * 50)
    print('Iteration', iteration)
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print(q[::-1] if INVERT else q, '(%s)' % correct, '-', guess)


Train on 35937 samples, validate on 3992 samples
Epoch 1/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0158 - acc: 0.9963 - val_loss: 0.0628 - val_acc: 0.9835
Epoch 2/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0157 - acc: 0.9964 - val_loss: 0.0626 - val_acc: 0.9839
Epoch 3/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0155 - acc: 0.9965 - val_loss: 0.0624 - val_acc: 0.9836
Epoch 4/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0151 - acc: 0.9966 - val_loss: 0.0622 - val_acc: 0.9838
Epoch 5/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0150 - acc: 0.9966 - val_loss: 0.0624 - val_acc: 0.9839
Epoch 6/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0150 - acc: 0.9966 - val_loss: 0.0625 - val_acc: 0.9838
Epoch 7/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0146 - acc: 0.9968 - val_loss: 0.0624 - val_acc:

35937/35937 [==============================] - 2s 45us/step - loss: 0.0114 - acc: 0.9979 - val_loss: 0.0621 - val_acc: 0.9842
Epoch 6/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0113 - acc: 0.9978 - val_loss: 0.0619 - val_acc: 0.9842
Epoch 7/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0113 - acc: 0.9978 - val_loss: 0.0622 - val_acc: 0.9844
Epoch 8/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0111 - acc: 0.9979 - val_loss: 0.0624 - val_acc: 0.9841
Epoch 9/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0110 - acc: 0.9980 - val_loss: 0.0619 - val_acc: 0.9846
Epoch 10/10
35937/35937 [==============================] - 2s 45us/step - loss: 0.0111 - acc: 0.9979 - val_loss: 0.0627 - val_acc: 0.9842

--------------------------------------------------
Iteration 193
                       culiacan (culiacans                       ) - culiacans                       
                    

35937/35937 [==============================] - 2s 45us/step - loss: 0.0090 - acc: 0.9985 - val_loss: 0.0622 - val_acc: 0.9851

--------------------------------------------------
Iteration 196
                         oilman (oilmen                          ) - oilmen                          
                  butterfingers (butterfinger                    ) - butterfingrr                    
                        chapter (chapters                        ) - chapters                        
                 nibelungenlied (nibelungenlieds                 ) - nieblengeeeeees                 
                          talus (taluses                         ) - taluses                         
                        rydberg (rydbergs                        ) - rydbergs                        
                        limiter (limiters                        ) - limiters                        
                      hyperpnea (hyperpneas                      ) - hyperpheas               

In [19]:
for i in range(10):
    ind = np.random.randint(0, len(x_val))
    rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print(q[::-1] if INVERT else q, '(%s)' % correct, '-', guess)

                        hodeida (hodeidas                        ) - hodeidas                        
                         redbud (redbuds                         ) - redbuds                         
                        fitment (fitments                        ) - fitments                        
                       morrison (morrisons                       ) - morrisons                       
                           pung (pungs                           ) - pungs                           
                     micromeria (micromerias                     ) - micromerias                     
                        albumin (albumins                        ) - albumins                        
                        fetlock (fetlocks                        ) - fetlocks                        
                        sorbent (sorbents                        ) - sorbents                        
                         prague (pragues                         ) - pragues      